In [2]:
import pandas as pd
import os

In [3]:
def plot_function(particle_filter_list, beh_segmentation_list):
    
    list_of_avg = []
    x = 1

    # Iterate through the particle filter and beh_segmentation lists that contain the absolute paths of the files of interest.
    for (i, j) in zip(particle_filter_list, beh_segmentation_list):
        n_bins_per_seg = 10
        # for sess in SessionIterator("Miniscope recordings").findSessions():
        filtered = pd.read_hdf(i) # Particle Filter
        segmented = pd.read_hdf(j, key="/per_frame") # Behavioral Segmentation
        state_order_series = pd.Series(state_order, name="state_order")
        segmented_relevant = segmented.join(state_order_series, on="state_name")
        x_pos = segmented_relevant.spatial_progress + segmented_relevant.state_order
        speeds = filtered.speed[np.isfinite(x_pos)]
        bin_x = np.floor(x_pos[np.isfinite(x_pos)]*n_bins_per_seg).astype(int)
        gby = (60*speeds).groupby([bin_x, segmented_relevant.segment_no]).mean().groupby(level=0)
        avgs = gby.mean()
        low_q = gby.quantile(0.25)
        high_q = gby.quantile(0.75)
        N_trials = len(segmented[segmented.state_name == "leftReward"].segment_no.unique())
        avgs = avgs.clip(lower=0)
        list_of_avg += [avgs]
        
        # Plotting
        plt.figure(figsize=(7, 3))
        plt.plot(avgs)
        plt.fill_between(avgs.index, low_q, high_q, alpha=.2)
        plt.text(0.9, 0.85, f"N={N_trials} trials", ha="right", transform=plt.gca().transAxes)
        plt.title("Animal "+str(x)+" learning day 1")
        labels = ["Turn at initiation", "Run in main arm", "Turn to left", "Run in left arm", "At reward"]
        for i in range(1, 5):
            plt.axvline(i*n_bins_per_seg, color='k', linestyle='--')
        plt.xticks((np.arange(5)+0.5)*n_bins_per_seg, labels, rotation=15, ha="right")
        plt.ylabel("Average speed (cm/s)")
        plt.ylim(-5, 30)
        sns.despine()
        print(i, j)
        x += 1

        # Saving plots
        plt.savefig('/Users/pierre.le.merre/OneDrive - KI.SE/Mac/Desktop/animals1-2-3-learningday1&11/'+'learningday'+str(x)+'.png', bbox_inches='tight', dpi=300 )
        plt.show()
